# Módulo 2 - Técnicas de Programação II
## Aula 6
- Gráficos com Plotly
- Exercícios

In [ ]:
!pip install plotly
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Exercício
Agora é hora de praticar!  
  
Faremos os mesmos exercícios praticados na aula passada, porém agora com o Plotly, para verificarmos as diferenças entre ambos.
  
Os bancos obtêm uma receita importante com empréstimos concedidos. Mas muitas vezes está associado ao risco. O mutuário pode não pagar o empréstimo. Para mitigar esse problema, os bancos decidiram usar o Machine Learning para superar esse problema. Eles coletaram dados anteriores sobre os tomadores de empréstimos e gostariam que você desenvolvesse um modelo de ML forte para classificar se algum novo devedor provavelmente entrará em default ou não.  
  
O conjunto de dados é enorme e consiste em vários fatores determinísticos, como renda do mutuário, gênero, finalidade do empréstimo, etc. O conjunto de dados está sujeito a uma forte multicolinearidade e valores vazios. Você pode superar esses fatores e construir um classificador forte para prever inadimplentes?

Importe a base **Loan_Default.csv** que está em csv dentro do diretório **data**.

In [ ]:
link = 'https://drive.google.com/open?id=12LpCQ8fVeOKHKWChFDVohDfU9nAL-eTy' # The shareable link
##

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.

def import_drive_file(link, sep):
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    fluff, id = link.split('=')
    print (id) # Verify that you have everything after '='

    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('Filename.csv')  
    df = pd.read_csv('Filename.csv', sep=sep)
    return df

In [ ]:
df_loan = import_drive_file(link="https://drive.google.com/open?id=1GbY4Hd8gSZ7s-_UNW13a-IMue3vQlmEF", sep=",")

1GbY4Hd8gSZ7s-_UNW13a-IMue3vQlmEF


In [ ]:
df_loan.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,construction_type,occupancy_type,income,credit_type,Credit_Score,age,submission_of_application,LTV,Region,Status
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,sb,pr,1740.0,EXP,758,25-34,to_inst,98.728814,south,1
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,sb,pr,4980.0,EQUI,552,55-64,to_inst,NaN,North,1
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,sb,pr,9480.0,EXP,834,35-44,to_inst,80.019685,south,0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,sb,pr,11880.0,EXP,587,45-54,not_inst,69.376900,North,0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,sb,pr,10440.0,CRIF,602,25-34,not_inst,91.886544,North,0


Verifique com a função .info os campos presentes na tabela

In [ ]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  interest_only              148670 non-null  object 
 13  lump_sum_payment           14

Imprima as estatísticas padrões do dataframe com a função .describe

In [ ]:
df_loan.describe()

,ID,year,loan_amount,rate_of_interest,property_value,income,Credit_Score,LTV,Status
count,148670.000000,148670.0,1.486700e+05,112231.000000,1.335720e+05,139520.000000,148670.000000,133572.000000,148670.000000
mean,99224.500000,2019.0,3.311177e+05,4.045476,4.978935e+05,6957.338876,699.789103,72.746457,0.246445
std,42917.476598,0.0,1.839093e+05,0.561391,3.599353e+05,6496.586382,115.875857,39.967603,0.430942
min,24890.000000,2019.0,1.650000e+04,0.000000,8.000000e+03,0.000000,500.000000,0.967478,0.000000
25%,62057.250000,2019.0,1.965000e+05,3.625000,2.680000e+05,3720.000000,599.000000,60.474860,0.000000
50%,99224.500000,2019.0,2.965000e+05,3.990000,4.180000e+05,5760.000000,699.000000,75.135870,0.000000
75%,136391.750000,2019.0,4.365000e+05,4.375000,6.280000e+05,8520.000000,800.000000,86.184211,0.000000
max,173559.000000,2019.0,3.576500e+06,8.000000,1.650800e+07,578580.000000,900.000000,7831.250000,1.000000


### Gráficos

1) Verifique sua variável de interesse (Status) como a contagem de distribuição

In [ ]:
px.bar(df_loan["Status"].value_counts())

2) Construa um gráfico de barras que permita verificar a Bad Rate (média da coluna Status) por tipo de crédito oferecido (credit_type). 

In [ ]:
df_bar = df_loan.pivot_table(index='credit_type', values='Status', aggfunc='mean')
fig = px.bar(df_bar)
fig.update_layout(
    xaxis_title="Credit Type",
    yaxis_title="Bad Rate",
    title={
        'text': 'Bad Rate by Credit Type',
        'y':0.95,
        'x':0.5,
        'font_color': 'grey'
    })
fig.show()

3) Faça um gráfico de dispersão da Credit Score pela média do Status

In [ ]:
df_credit_score = df_loan.pivot_table(index='Credit_Score', values='Status', aggfunc='mean').reset_index()

fig=px.scatter(data_frame=df_credit_score, x='Credit_Score', y='Status', marginal_x='box', marginal_y='box')
fig.update_layout(
    xaxis_title={
        'text':'Credit Score',
        'font_size': 18
    },
    yaxis_title={
        'text': 'Bad Rate',
        'font_size': 18
    },
    font_color='grey',
    title={
        'text': 'Bad Rate by Credit Score',
        'x': 0.5,
        'y': 0.93,
        'font_family': 'Times New Roman',
        'font_color': 'black',
        'font_size': 20
    },
    template='ggplot2'
)

4) Por meio de um gráfico de dispersão, identifique o comportamento da média da variável de interesse (Status) por faixa etária

In [35]:
df_age= df_loan.pivot_table(index='age', values='Status', aggfunc='mean').reset_index()
df_age

,age,Status
0,25-34,0.221920
1,35-44,0.222713
2,45-54,0.240467
3,55-64,0.258868
4,65-74,0.268560
5,<25,0.289454
6,>74,0.300070


In [36]:
fig=px.scatter(df_age, x="age", y="Status")
fig.update_xaxes(categoryorder='array', categoryarray= ['<25','25-34','35-44','45-54','55-64', '65-74','>74'])

4) Construa o histograma do montante de dívida (loan_amount) e verifique como está sua distribuição. Observe o mesmo com um gráfico boxplot.

In [37]:
px.histogram(df_loan["loan_amount"], marginal="box")

In [ ]:
fig=px.box(data_frame=df_loan, x='loan_amount', orientation='h')

fig.update_layout(
    xaxis_title='Boxplot of the Loan Amount',
    title={
        'text': 'Boxplot',
        'x': 0.49,
        'y': 0.94,
        'font_color': 'grey'
    },
    template='none'
)

5) Como se encontra a distribuição das faixas-etárias nesta base de dados.  
*Obs.: utilize o gráfico de pizza*

In [7]:
df_pizza=df_loan.pivot_table(index='age', values='ID', aggfunc='count')

fig=px.pie(names=df_pizza.index, values=df_pizza['ID'], opacity=0.8)

fig.update_layout(
    title={
        'text': 'Distribuição das Idades',
        'x':0.5,
        'y':0.94
    }
)


**Extra**: Primeiramente imagine uma hipótese de relação entre idade, renda e score de crédito dos indivíduos presenes neste dataframe. Agora, com auxílio gráfico, entenda se a relação que existe está dentro da hipótese ou não.

In [33]:
df_h = df_loan.pivot_table(index="age", values=['Credit_Score', "income"], aggfunc='mean').reset_index()
fig = px.bar(df_h, x="age", y=["Credit_Score","income"], barmode='group')

fig.update_layout(
    xaxis_title="Faixa Etária",
    yaxis_title="Credit Score e Renda (Mean)",
    legend_title="Legenda",
    template='none',
    title={
        'text': 'Média da Renda e do Credit Score pela Idade',
        'y':0.95,
        'x':0.5,
        'font_color': 'grey'
    }
)
fig.update_xaxes(categoryorder='array', categoryarray= ['<25','25-34','35-44','45-54','55-64', '65-74','>74'])


In [ ]:
pra quem quer levar o matplot lib para um outro patamar, achei sensacional os gráficos feitos nesse projeto:
https://www.kaggle.com/code/joshuaswords/netflix-data-visualization/notebook